In [82]:
import pandas as pd
from datetime import datetime
from keras.models import Sequential
from keras.layers import Dense
import tensorflow as tf
import math
import statistics
import keras
from tensorflow.python.ops import math_ops
import numpy as np


In [123]:
dfs = pd.read_csv('./dfs_all_with_sincos_8h_pojutrze.csv')
dfs = dfs.iloc[: , 1:]

In [124]:
df_shuffle = dfs.sample(frac = 1)

train_length = math.floor(df_shuffle.shape[0] * 0.8)

Xs = df_shuffle.iloc[: , :-1]
X_norm = Xs
X_norm = (Xs-Xs.mean())/Xs.std()
X_norm = X_norm.replace(np.nan, 0)


x_test = X_norm.iloc[train_length: , :]
y_test = df_shuffle.iloc[train_length: , -1:]
x_train = X_norm.iloc[:train_length , :]
y_train = df_shuffle.iloc[:train_length , -1:]

Regresja MSE temperatura 5000 danych

In [126]:
def correctly_predicted(predicted, y_test):
  counter = 0
  for i in range(len(predicted)):
    if abs(predicted[i] -  y_test[i]) <= 2:
      counter+=1

  return counter

def test_loss_function(loss):
  dim = x_test.shape[1]
  lr_schedule = tf.keras.optimizers.schedules.ExponentialDecay(
      initial_learning_rate=0.1,
      decay_steps=100,
      decay_rate=0.9)
  adam = tf.keras.optimizers.Adam(
      learning_rate=lr_schedule)
  # adam = tf.keras.optimizers.Adam(
  #     learning_rate=0.00000001)
  model = Sequential()
  model.add(Dense(12, input_dim=dim, activation='relu'))
  model.add(Dense(10, activation='relu'))
  model.add(Dense(10, activation='relu'))
  model.add(Dense(1, activation='relu'))
  model.compile(loss=loss, optimizer=adam)
  model.fit(x=x_train, y=y_train, epochs=50, verbose=0)
  # model.fit(x=x_train, y=y_train, epochs=50)
  predicted = model.predict(x_test)
  return predicted

In [127]:
losses = [
          tf.keras.losses.MeanSquaredError(),
          tf.keras.losses.MeanAbsoluteError(),
          tf.keras.losses.MeanAbsolutePercentageError(),
          tf.keras.losses.MeanSquaredLogarithmicError(),
          tf.keras.losses.CosineSimilarity(),
          tf.keras.losses.Huber(),
          tf.keras.losses.LogCosh(),
          ]

predictions = []

for loss in losses:
  predictions.append(test_loss_function(loss))
  print(type(loss), ' finished')

for i in range(len(predictions)):
  correct_predictions = correctly_predicted(predictions[i].flatten(),  y_test.iloc[:,0].tolist())
  print(type(losses[i]), '    ', correct_predictions, correct_predictions / len(predictions[i]))

<class 'keras.losses.MeanSquaredError'>  finished
<class 'keras.losses.MeanAbsoluteError'>  finished
<class 'keras.losses.MeanAbsolutePercentageError'>  finished
<class 'keras.losses.MeanSquaredLogarithmicError'>  finished
<class 'keras.losses.CosineSimilarity'>  finished
<class 'keras.losses.Huber'>  finished
<class 'keras.losses.LogCosh'>  finished
<class 'keras.losses.MeanSquaredError'>      5063 0.46351734871372335
<class 'keras.losses.MeanAbsoluteError'>      5360 0.49070768104000734
<class 'keras.losses.MeanAbsolutePercentageError'>      5445 0.4984894259818731
<class 'keras.losses.MeanSquaredLogarithmicError'>      2575 0.2357410967682871
<class 'keras.losses.CosineSimilarity'>      0 0.0
<class 'keras.losses.Huber'>      5455 0.4994049253867985
<class 'keras.losses.LogCosh'>      5207 0.4767005401446489


In [ ]:
def custom_loss(y_true, y_pred):
  counter = 0
  for i in range(len(y_pred)):
    if abs(y_pred[i] -  y_true[i]) > 2:
      counter+=1
  return counter/len(y_pred)

In [ ]:
df_class_shuffle = dfs_class.sample(frac = 1)

train_class_length = math.floor(df_class_shuffle.shape[0] * 0.8)

Xs = df_class_shuffle.iloc[: , :-1]
X_norm = Xs
X_norm = (Xs-Xs.mean())/Xs.std()

x_test = X_norm.iloc[train_length: , 2:]
y_test = df_class_shuffle.iloc[train_length: , -1:]
x_train = X_norm.iloc[:train_length , 2:]
y_train = df_class_shuffle.iloc[:train_length , -1:]

#zamiana y na {0,1}
y_train.loc[y_train['WindSpeed5'] < 6.0, 'WindSpeed5'] = 0
y_train.loc[y_train['WindSpeed5'] >= 6.0, 'WindSpeed5'] = 1
y_test.loc[y_test['WindSpeed5'] < 6.0, 'WindSpeed5'] = 0
y_test.loc[y_test['WindSpeed5'] >= 6.0, 'WindSpeed5'] = 1

In [ ]:
print(y_train.head(20))

In [ ]:
dim = x_test.shape[1]
model = Sequential()
model.add(Dense(12, input_dim=dim, activation='sigmoid'))
model.add(Dense(10, activation='sigmoid'))
model.add(Dense(10, activation='sigmoid'))
model.add(Dense(1, activation='sigmoid'))
model.compile(optimizer='adam', loss='binary_crossentropy',
              metrics=['accuracy'])

In [ ]:
model.fit(x=x_train, y=y_train, epochs=50)